In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
import os,sys, kimimaro
import numpy as np
from skimage.morphology import disk, ball, square
import scipy.ndimage as nd
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from IPython.display import HTML

sys.path.append("/home/phornauer/Git/axon_tracking/")
from axon_tracking import skeletonization as skel

In [ ]:
root_path = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/iNeurons/230817/T002443/AxonTracking/'
stream_id = 'well006'
template_id = 262

template = skel.load_template_file(root_path, stream_id, template_id)

In [ ]:
root_path = "/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/iNeurons/230731/T002443/AxonTracking/"
stream_id = 'well009'
template_id = 189
template = skel.load_template_file(root_path, stream_id, template_id)

In [ ]:
root_path = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/GNM24/230511/T002441/AxonTracking/'
stream_id = 'well007'
template_id = 12

template = skel.load_template_file(root_path, stream_id, template_id)

In [ ]:
dilation_params = dict()
dilation_params['r_dilation'] = 2 #Distance in electrodes to detect new peaks
dilation_params['init_th'] = -5 #Minimum peak amplitude to be a seed point (initial detection)
dilation_params['min_th'] = -0.2 #Minimum peak amplitude to be eligible for peak detection
dilation_params['filter_footprint'] = np.ones((3,3,2)) #Median filter footprint (x*y*time)
dilation_params['use_derivative'] = False #Use derivative of template for detection

In [ ]:
capped_matrix, target_coor, post_coor = skel.localize_neurons(template, ms_cutout=[1.5, 5.0])

In [ ]:
input_mat = template
v_min = -5
v_max = 0

tmp_filt = nd.median_filter(input_mat,footprint=ball(1))#size=(3,3,2))

In [ ]:
#tmp_filt = nd.median_filter(template,footprint=ball(1))
#tmp_filt = nd.rank_filter(template,rank=-26,footprint=ball(2))


#tmp_filt = nd.percentile_filter(input_mat,percentile=20,footprint=ball(2))
fig, axes = plt.subplots(1,2)
plt.subplot(121)
plt.imshow(np.min(input_mat,axis=2).T,vmin=v_min, vmax=v_max)

ax = plt.subplot(122)
im = plt.imshow(np.min(tmp_filt,axis=2).T,vmin=v_min, vmax=v_max)
#fig.colorbar(im, ax=axes.ravel().tolist(),shrink =0.275, label="Amplitude [uV]")

plt.show()

In [ ]:
tmp_interp = skel.interpolate_template(tmp_filt,spacing=1/2)

In [ ]:
ani = skel.generate_propagation_gif(tmp_interp,downsample=1,clim=[-10,0])

In [ ]:
ani.save("/home/phornauer/neuron+skeleton.gif")

In [ ]:
#tmp_filt = nd.percentile_filter(template,percentile=20,footprint=ball(2))
filtered = skel.iterative_dilation(tmp_interp, **dilation_params)

In [ ]:
dil_idx = np.argwhere(filtered)

x = dil_idx[:,0]
y = dil_idx[:,1]
z = dil_idx[:,2]
fig = plt.figure(figsize=(8,4))
#vals = capped_input[tuple(dil_idx.T)]
im = plt.scatter(x,tmp_interp.shape[1]-y,s=5,c=z,cmap="magma")
fig.colorbar(im,label="Peak latency [ms]")
plt.xlim([0, tmp_interp.shape[0]])
plt.ylim([0, tmp_interp.shape[1]])
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(projection='3d')
im = ax.scatter3D(x,y,z,s=5,c=z,cmap="viridis")
ax.view_init(10,40)
plt.xlim([0, 220])
plt.ylim([0, 120])
plt.xlabel("x")
plt.ylabel("y")
plt.colorbar(im, label="Peak latency [ms]",shrink=0.7)
plt.show()

In [ ]:
skel_params = dict()
skel_params['scale'] = 0.5
skel_params['const'] = 15
skel_params['pdrf_exponent'] = 4
skel_params['pdrf_scale'] = 10
skel_params['dust_threshold'] = 100
skel_params['anisotropy'] = (10.0, 10.0, 10.0)
skel_params['tick_threshold'] = 10

skels = skel.skeletonize(filtered, **skel_params)
skeleton = skels[1]
#skeleton = kimimaro.join_close_components(skels[1], radius=1000)

fig = plt.figure(figsize=(8,4))
x,y,z = [skeleton.vertices[:,x] for x in range(3)]
im = plt.scatter(x,tmp_interp.shape[1]-y,c=z,s=5,cmap="coolwarm")
fig.colorbar(im, label='Peak latency [samples]')
#plt.scatter(target_coor[0],target_coor[1],c=z[target_coor[2]],s=100)
plt.xlim([0, tmp_interp.shape[0]])
plt.ylim([0, tmp_interp.shape[1]])
plt.show()

In [ ]:
skeleton

In [ ]:
ani = skel.generate_propagation_gif(tmp_interp, cumulative=False, skeleton = skeleton, downsample=2, clim=[-5, 0], cmap="Greys")

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(projection='3d')
ax.scatter3D(x,y,z,s=10,c=z,cmap="magma")
#ax.scatter3D(target_coor[0],target_coor[1],target_coor[2],s=100)
ax.view_init(0,140)
ax.set
plt.xlim([0,220])
plt.ylim([0,120])

plt.show()

In [ ]:
source = skeleton.vertices[skeleton.edges[:,0],0:2]
target = skeleton.vertices[skeleton.edges[:,1],0:2]
segs = [np.vstack((source[x], target[x])) for x in range(source.shape[0])]

In [ ]:
from matplotlib.collections import LineCollection
fig, ax = plt.subplots()
ax.set_xlim(0, 220)
ax.set_ylim(0, 120)
line_segments = LineCollection(segs, linestyle='solid',colors=[0,0,0])
ax.add_collection(line_segments)
plt.scatter(x,y,c=z,s=6,cmap="magma")
plt.show()